In [ ]:
!gdown --id 1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
!gdown --id 19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
!gdown --id 1ol6w0-OaxEe74GBVRP2edb0iBsOViV8i # film_adaptations

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NM4uSjSg0e2LPBT_aC0v1XaRRh932jxJ
To: /content/books.csv
100% 69.0M/69.0M [00:01<00:00, 48.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19IDNSFnAh-JiVx05BSJJRmU-eTdzlG1T
To: /content/films.csv
100% 28.6M/28.6M [00:00<00:00, 49.9MB/s]


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.1 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 
     |████████████████████████████████| 182 kB 59.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
from collections import defaultdict
from typing import Dict, List
import numpy as np


In [ ]:
books = pd.read_csv('books.csv')
films = pd.read_csv('films.csv')

In [ ]:
books = books[~books.isna().any(axis=1)].reset_index(drop=True)
films = films[~films.isna().any(axis=1)].reset_index(drop=True)

In [ ]:
# list_of_idx_books = [
#     99, 1980, 10618, 18685, 33857, 54081, 17357, 659, 35770
# ]

# books = books[books.id.isin(list_of_idx_books)]

In [ ]:
films.shape

(23120, 7)

In [ ]:
books.shape

(57588, 6)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
embed_bert_cls("привет как дела", model, tokenizer).shape

(312,)

In [ ]:
books_small = books.copy()
films_small = films.copy()

In [ ]:
films_small.shape

(23120, 7)

In [ ]:
books_small_emb = books_small.annotation.apply(lambda x: embed_bert_cls(x, model, tokenizer))
films_small_emb = films_small.description.apply(lambda x: embed_bert_cls(x, model, tokenizer))

In [ ]:
books_small_emb = np.stack(books_small_emb.values)
films_small_emb = np.stack(films_small_emb.values)

In [ ]:
book_embeddings = pd.concat([
    books_small[['id', 'title', 'authors']],
    pd.DataFrame(books_small_emb.astype('float16'), columns=[f'emb_book_{i}' for i in range(312)])
], axis=1)

film_embeddings = pd.concat([
    films_small[['id', 'title', 'year']],
    pd.DataFrame(films_small_emb.astype('float16'), columns=[f'emb_film_{i}' for i in range(312)])
], axis=1)

book_embeddings.to_csv('description_emb_books.csv', index=False)
film_embeddings.to_csv('description_emb_films.csv', index=False)

In [61]:
# list_of_idx_books = [
#     99, 1980, 10618, 18685, 33857, 54081, 17357, 659, 35770
# ]
list_of_idx_books = book_embeddings['id'].sample(7000, random_state=42).tolist()

book_embeddings_ = book_embeddings[book_embeddings.id.isin(list_of_idx_books)]

In [63]:
recommendations = []
N = 7


for idx, emb_one_book in tqdm(zip(list_of_idx_books, books_small_emb), total=len(list_of_idx_books)):
    sim_coefs = cosine_similarity(emb_one_book.reshape(1, -1), films_small_emb)

    recommender_idxs_by_book = [
        sorted(range(len(sim_coef)), key=lambda sub: sim_coef[sub], reverse=True)[:N]
        for sim_coef in sim_coefs
    ]

    book = books_small[books_small.id == idx]
    films_rec = films_small[films_small.index.isin(recommender_idxs_by_book[0])]

    for film in films_rec.iterrows():
        recommendations.append([
            book['id'].tolist()[0],
            book.title.tolist()[0],
            book.genres.tolist()[0],
            film[1]['id'],
            film[1]['title'],
            film[1]['genres'],
        ])

recommendations = pd.DataFrame(recommendations, columns=['book_id', 'book_title', 'book_genres', 'film_id', 'film_title', 'film_genre'])

100%|██████████| 7000/7000 [06:50<00:00, 17.06it/s]


## Проверка с помощью метрики по экранизациям

In [42]:
film_adaptations = pd.read_csv('film_adaptations.csv')

In [76]:
def IoU(films_rec: list, films_match: list):
    films_rec = set(films_rec)
    films_match = set(films_match)
    intersection = films_rec.intersection(films_match)
    max_length = max(len(films_rec), len(films_match))

    return len(intersection) / max_length


def metric_film_adaptations(
    recommendations: pd.DataFrame,
    matching: pd.DataFrame
) -> float:
    """
    This metric is calculated based on movie recommendations based on books.
    We want to know if we have a movie adaptation of the book in our recommendation.

    How to calculate:
      If there is a film adaptation of the book, we put a match, otherwise there 
      is no match. If there aren't any film adaptations we add 0.5 to result.
      Next, we average the result

    Explanation of the metric:
      This metric like a roc_auc. If metric > 0.5, it is good, otherwise it is bad XD

    Input:
      recommendations: Dataframe with book id as a key and movie ids as value
      matching: Dataframe that contains a book id and a movie id.
                Has to have 'id_book' and 'id_film' columns

    Return:
      metric value
    """

    assert 'book_id' in set(recommendations.columns) and \
      'film_id' in set(recommendations.columns) and \
      'book_id' in set(matching.columns) and \
      'film_id' in set(matching.columns), \
      "matching doesn't contain right columns, please rename them to id_film and id_book"

    matching = (
        matching
        .groupby(['book_id'])['film_id']
        .apply(list)
        .reset_index(name='film_id')
    )
    recommendations = (
        recommendations
        .groupby(['book_id'])['film_id']
        .apply(list)
        .reset_index(name='film_id')
    )
    

    book_intersection = recommendations.merge(
        matching, on='book_id', how='inner', suffixes=('_rec', '_match')
    )

    print('intersection of datasets is - ', len(book_intersection))

    book_intersection = book_intersection.apply(lambda x: IoU(x['film_id_rec'], x['film_id_match']), axis=1)
    result = sum(book_intersection) / book_intersection.shape[0] # .astype(bool).astype(int)


    return result

In [77]:
metric_film_adaptations(recommendations, film_adaptations)

intersection of datasets is -  178


0.0

In [80]:
metric_film_adaptations(film_adaptations, film_adaptations)

intersection of datasets is -  1525


1.0

In [78]:
recommendations[recommendations.book_id == 58158]

,book_id,book_title,book_genres,film_id,film_title,film_genre
14364,58158,Кроткая,"Литература 19 века,Русская классика",6040,Господин Никто,"драмы, зарубежные, мелодрамы, фантастика, фэнтези"
14365,58158,Кроткая,"Литература 19 века,Русская классика",9593,"Скажи, что любишь меня","драмы, мелодрамы"
14366,58158,Кроткая,"Литература 19 века,Русская классика",11252,Любовь случается,"драмы, зарубежные, мелодрамы"
14367,58158,Кроткая,"Литература 19 века,Русская классика",12185,Пена дней,"экранизации, драмы, зарубежные, мелодрамы, фэн..."
14368,58158,Кроткая,"Литература 19 века,Русская классика",14592,Газели,"зарубежные, комедии"
14369,58158,Кроткая,"Литература 19 века,Русская классика",15983,Откровения,"драмы, зарубежные"
14370,58158,Кроткая,"Литература 19 века,Русская классика",17739,Огни притона,драмы


In [79]:
film_adaptations[film_adaptations.book_id == 58158]

,book_title,book_id,book_authors,film_title,film_id,film_year,film_rating
1380,Кроткая,58158,Федор Достоевский,Кроткая,18852,1960.0,7.5


## Проверка с помощью метрики по жанрам

In [83]:
!gdown 1lsrSjBQyY8PaJXGzp5Sr883loNOjND-C # books
!gdown 1WrlB_ZtJhZth4UdyA6S3GlE4YnXtquuv # films

Downloading...
From: https://drive.google.com/uc?id=1lsrSjBQyY8PaJXGzp5Sr883loNOjND-C
To: /content/books_with_lemmas.csv
100% 78.7M/78.7M [00:00<00:00, 165MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WrlB_ZtJhZth4UdyA6S3GlE4YnXtquuv
To: /content/films_with_lemmas.csv
100% 33.7M/33.7M [00:00<00:00, 222MB/s]


In [84]:
import pandas as pd

books = pd.read_csv('books_with_lemmas.csv', index_col=0)
books['lemmas_inter'] = books['lemmas_inter'].apply(lambda x: string_to_set(x))
books['id'] = books['id'].astype(int)

films = pd.read_csv('films_with_lemmas.csv', index_col=0)
films['lemmas_inter'] = films['lemmas_inter'].apply(lambda x: string_to_set(x))
films['id'] = films['id'].astype(int)


def string_to_set(string):
    res = set(string.replace('{', '').replace('}', '').replace('\\', '').replace("'", '').replace('"', '').strip().split(','))
    clear_res = set()
    for elem in res:
        clear_res.add(elem.strip())
    return clear_res


class BooksToFilmsMetrics:

    def accuracy(self, books: pd.DataFrame, films: pd.DataFrame) -> pd.DataFrame:

        books['key'] = 0
        films['key'] = 0

        df = books.merge(films, on='key', how='outer')
        df = df[['id_x', 'id_y', 'lemmas_inter_x', 'lemmas_inter_y']]
        df.columns = ['book_id', 'film_id', 'book_lemmas', 'film_lemmas']

        df['metric'] = df.apply(lambda row: len(row['film_lemmas'].intersection(row['book_lemmas'])) / len(row['book_lemmas']), axis=1)
        return df[['book_id', 'film_id', 'metric']]

    def weighted_accuracy(self, books: pd.DataFrame, films: pd.DataFrame, book_weight: float = 1, film_weight: float = 2.5) -> list:
      
        books['key'] = 0
        films['key'] = 0

        df = books.merge(films, on='key', how='outer')
        df = df[['id_x', 'id_y', 'lemmas_inter_x', 'lemmas_inter_y']]
        df.columns = ['book_id', 'film_id', 'book_lemmas', 'film_lemmas']

        df['metric'] = df.apply(lambda row: len(row['film_lemmas'].intersection(row['book_lemmas'])) /\
                                (book_weight*len(row['book_lemmas']) + film_weight*len(row['film_lemmas'])), axis=1)
        return df[['book_id', 'film_id', 'metric']]
    
    def absolute_accuracy(self, books: pd.DataFrame, films: pd.DataFrame, l1: float = 1.0) -> list:
        books['key'] = 0
        films['key'] = 0

        df = books.merge(films, on='key', how='outer')
        df = df[['id_x', 'id_y', 'lemmas_inter_x', 'lemmas_inter_y']]
        df.columns = ['book_id', 'film_id', 'book_lemmas', 'film_lemmas']

        df['metric'] = df.apply(lambda row: (len(row['film_lemmas'].intersection(row['book_lemmas'])) - l1*len(row['book_lemmas'])) / len(row['book_lemmas']), axis=1)
        return df[['book_id', 'film_id', 'metric']]


In [107]:
genres_metric = BooksToFilmsMetrics()
result = []

for book_id in tqdm(recommendations.book_id.unique()):
    result.append(genres_metric.accuracy(
        books[books['id'] == book_id].head(1),
        films[films['id'].isin(recommendations[recommendations.book_id == book_id].film_id.tolist())]
    ))

  0%|          | 0/7000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  3%|▎         | 190/7000 [00:02<02:22, 47.86it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 7000/7000 [00:54<00:00, 127.79it/s]


In [109]:
print("Metric by genres =", np.mean([recs_iou.metric.mean() for recs_iou in result]))

Metric by genres = 0.04919914072522115


## Проверка с помощью небольшой ручной разметки

In [87]:
!gdown 1JOmJxF__DIRr-by83sIEGcrEEBCxhsKG

class Metrics:
    def __init__(self, gold_rec, new_rec):
        self.gold_rec = gold_rec[["book_title", "film_title", "rank"]]
        self.new_rec = new_rec[["book_title", "film_title"]]

    def metric_set(self):
        ans = {"book_title": [], "similarity": []}

        gr_gold = self.gold_rec.groupby(by=["book_title"], sort=False)[
            "film_title"
        ].agg(size=len, set=lambda x: set(x))
        gr_gold["book_title"] = gr_gold.index
        gr_new = self.new_rec.groupby(by=["book_title"], sort=False)["film_title"].agg(
            size=len, set=lambda x: set(x)
        )
        gr_new["book_title"] = gr_new.index

        books_list = gr_new["book_title"].tolist()
        for i in range(len(books_list)):

            book = books_list[i]
            new_data = gr_new[gr_new["book_title"] == book][["size", "set"]].values[0]
            try:
                gold_data = gr_gold[gr_gold["book_title"] == book][
                    ["size", "set"]
                ].values[0]
            except IndexError:
                ans["book_title"].append(book)
                ans["similarity"].append("-")
                continue

            similarity = len(gold_data[1] & new_data[1]) / gold_data[0]
            ans["book_title"].append(book)
            ans["similarity"].append(similarity)

        return pd.DataFrame(ans)

    def metric2(self):
        pass

Downloading...
From: https://drive.google.com/uc?id=1JOmJxF__DIRr-by83sIEGcrEEBCxhsKG
To: /content/gold_authors.csv
100% 22.9k/22.9k [00:00<00:00, 25.6MB/s]


In [88]:
gold_rec = pd.read_csv('gold_authors.csv')

In [116]:
books_for_metric = books.drop_duplicates(keep='first', subset=['title'])[['id', 'title']]

In [118]:
gold_rec = gold_rec.merge(books_for_metric, how='left', left_on='book_title', right_on='title')

In [120]:
metrics = Metrics(gold_rec=gold_rec, new_rec=recommendations)

In [122]:
result_by_handle_recoms = metrics.metric_set()

In [124]:
result_by_handle_recoms.similarity.value_counts()

-      6921
0.0      15
Name: similarity, dtype: int64

In [126]:
result_by_handle_recoms

,book_title,similarity
0,По ту сторону реальности. Необычное наследство...,-
1,Тиран,-
2,Финансовый гороскоп на 2021 год,-
3,Время остановилось,-
4,Мятежница,-
...,...,...
6931,Душман. Последний выстрел,-
6932,Юпитер поверженный,-
6933,«Тени» звёзд,-
6934,Таро. Полное руководство по чтению карт и пред...,-


In [128]:
recommendations[recommendations.book_title == 'Тиран']

,book_id,book_title,book_genres,film_id,film_title,film_genre
7,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",1899,Исход,"драмы, триллеры, военное"
8,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",3106,Красный призрак,"русские, приключения, исторические, триллеры, ..."
9,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",15220,Неуловимые,"боевики, драмы"
10,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",16729,Живёшь только дважды,"боевики, триллеры, приключения"
11,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",16802,Последствия войны,"драмы, военные"
12,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",18125,Тёмная башня,"боевики, приключения, вестерн, фантастика, фэн..."
13,9910,Тиран,"Космическая фантастика,Любовно-фантастические ...",18389,Хеллбой II: Золотая армия,"боевики, приключения, фантастика, ужасы, фэнтези"
